In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
#from qat.core.console import display

In [ ]:
%load_ext qat.core.magic

#QPU connection
try:
    from qat.qlmaas import QLMaaSConnection
    connection = QLMaaSConnection()
    LinAlg = connection.get_qpu("qat.qpus:LinAlg")
    lineal_qpu = LinAlg()
except ImportError:
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()

In [ ]:
from qat.qpus import PyLinalg
lineal_qpu = PyLinalg()

In [ ]:
#Libreria Zalo
sys.path.append('/home/cesga/gferro/NEASQC/PhaseAmplification/')
#Libreria Juan
sys.path.append('/home/cesga/gferro/NEASQC/ProgramasDefinitivos/')

In [ ]:
def RunJob(job):
    result = lineal_qpu.submit(job)
    try:
        State = PostProcessResults(result.join())
    except AttributeError:
        State = PostProcessResults(result)
    return State

# Quantum Phase Amplification

Voy a comparar los operadores que monta Juan para la *Phase Amplification* con respecto a los que implementé yo.

El notebook *QPA04_PuertasBase.ipynb* tiene toda la teoría y la implementación de las diferentes puertas por lo que aquí voy a aligerar todo lo que pueda para hacer testeos rápidos.

## 1. Carga Completa

Lo primero que necesitamos es realizar una carga completa de los datos sobre el sitema cuántico. Esta operación se puede resumir del siguiente modo:

$$|\Psi\rangle_{n+1}=\hat{R_{n+1}}\left(\hat{P_{n}}\otimes I\right)|0\rangle_{n}\otimes|0\rangle_{1}$$


### 1.1 Preparo Datos

In [ ]:
from AuxiliarFunctions import  get_histogram, PostProcessResults, TestBins
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 4
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

### 1.2 Circuto de Carga de datos

In [ ]:
from QuantumMultiplexors_Module_examples import LoadingData

In [ ]:
qprog = LoadingData(p_X, f_X)
circuit = qprog.to_circ()
%qatdisplay circuit 
job = circuit.to_job()
InitialState = RunJob(job)

In [ ]:
InitialState.head()

## 2. Amplificación de Amplitud

Después de la carga de datos tenemos el sistema en el estado $|\Psi\rangle_{n+1}$ que se puede descomponer en dos estados ortogonales $|\Psi_{1}\rangle$ y $|\Psi_{0}\rangle$ del siguiente modo

$$|\Psi\rangle_{n+1}=\sqrt{a}|\Psi_{1}\rangle+\sqrt{1-a}|\Psi_{0}\rangle$$

Donde 

$$|\Psi_{1}\rangle = \sum_{x=0}^{2^{n}-1}|x\rangle_{n}\otimes\frac{1}{\sqrt{a}}\sqrt{p(x)f(x)}|1\rangle$$
$$|\Psi_{0}\rangle = \sum_{x=0}^{2^{n}-1}|x\rangle_{n}\otimes\frac{1}{\sqrt{1-a}}\sqrt{p(x)(1-f(x))}|0\rangle$$



Donde $a=\sum_{x=0}^{2^{n}-1}p(x)f(x)$

Es decir los $n$ primeros qbits están en una superposición de estados y lo único que los diferencia es el estado el último qbit!!

Como la integral es la amplitud del estado $|\Psi_{1}\rangle$ el paso siguiente es intentar maximizar la probabilidad de que al medir obtengamos dicho estado aplicando el algoritmo de Groover. 

El algoritmo de Groover adapatado a amplificación de fase consiste en aplicar un número óptimo de veces $k$ el operador $\hat{Q}$. Este operador se define como:

$$\hat{Q}=\hat{U}_{|\Psi\rangle} \hat{U}_{|\Psi_{0}\rangle}$$

Los operadores $\hat{U}_{|\Psi_{2}\rangle}$ y $\hat{U}_{|\Psi_{0}\rangle}$ se construyen del siguiente modo:

$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$$
$$\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle \langle \Psi|$$


### 2.1 Operador $\hat{U}_{|\Psi_{0}\rangle}$

Este operador se construye del siguiente modo:
$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$$

Una operación que sería circuitable del siguiente modo:

$$\hat{U}_{|\Psi_{0}\rangle }=(\hat{I_{n}}\otimes X)(\hat{I_{n}}\otimes Z)(\hat{I_{n}}\otimes X)$$

La aplicación de este operador sobre $|\Psi\rangle_{n+1}$ es:

$$\hat{U}_{|\Psi_{0}\rangle} |\Psi\rangle_{n+1} = \sqrt{a}|\Psi_{1}\rangle-\sqrt{1-a}|\Psi_{0}\rangle$$

Es decir el operador $\hat{U}_{|\Psi_{0}\rangle }$ realizaría una reflexión en torno el eje definido por el estado $|\Psi_{1}\rangle$


In [ ]:
from PhaseAmplification_Module import U_Phi_0 
from expectation_module import load_U0

In [ ]:
qZalo = LoadingData(p_X, f_X)
qZalo.apply(U_Phi_0(n_qbits+1), qZalo.registers)
circuitZ = qZalo.to_circ()
%qatdisplay circuitZ 
job = circuitZ.to_job()
ZaloState = RunJob(job)

In [ ]:
qJuan = LoadingData(p_X, f_X)
U0_gate = load_U0(n_qbits)
qJuan.apply(U0_gate, qJuan.registers)
circuitJ = qJuan.to_circ()
%qatdisplay circuitJ 
job = circuitJ.to_job()
JuanState = RunJob(job)

### Comparaciones 

In [ ]:
InitialState.head()

In [ ]:
ZaloState.head()

In [ ]:
JuanState.head()

Basicamente el operador de Juan implementa la operación inversa al mío (Zalo). Mientras en el caso de zalo se implementa una reflexión en torno al estado $|\Psi_{1}\rangle$ en el caso de Juan la reflexión es en torno al estado $|\Psi_{0}\rangle$.
Zalo cambia de signos todos los estados con qbit final $|0\rangle$ mientras Juan cambia de signo los estados con qbit final $|1\rangle$

### 2.2 Operador $\hat{U}_{|\Psi\rangle}$

El operador $\hat{U}_{|\Psi\rangle}$ se basa en el operador difusor de Groover. Y su forma es la siguiente:

$$\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle \langle \Psi|$$

Como

$$|\Psi\rangle_{n+1}=\hat{R_{n+1}}\left(\hat{P_{n}}\otimes I\right)|0\rangle_{n}\otimes|0\rangle_{1}$$

Entonces podremos componer el operador del siguiente modo:

$$\hat{U}_{|\Psi\rangle } =\hat{R_{n+1}}\left(\hat{P_{n}}\otimes I\right)\hat{D}_{0} \left(\hat{P_{n}}\otimes I\right)^{\dagger} \hat{R_{n+1}}^{\dagger}$$

Donde $\hat{D}_{0}$ es una reflexion entorno al estador **perpendicular** al estado $|0\rangle_{n}$

$$\hat{D}_{0} = \hat{I}-2|0\rangle \langle0|$$

#### 2.2.1 Implementación  $\hat{D}_{0}$

Se puede demostrar que la implementación Circuital del Operador $\hat{D}_{0}$ es:

$$\hat{D}_{0} = \hat{I}-2|0\rangle \langle0|= \hat{X}^{\otimes n} c^{n-1}Z \hat{X}^{\otimes n}$$

In [ ]:
from PhaseAmplification_Module import LoadD0_Gate

In [ ]:
D0 = LoadD0_Gate(n_qbits)

In [ ]:
%qatdisplay D0 --depth 1

In [ ]:
qD0 = LoadingData(p_X, f_X)
qD0.apply(LoadD0_Gate(n_qbits+1), qD0.registers)

In [ ]:
circuitD0 = qD0.to_circ()
%qatdisplay circuitD0 

In [ ]:
job = circuitD0.to_job()
D0State = RunJob(job)

In [ ]:
D0State.head()

In [ ]:
InitialState.head()

In [ ]:
(D0State['Amplitude'].loc[1:] == InitialState['Amplitude'].loc[1:]).all()

In [ ]:
(D0State['Amplitude'].loc[0] == -InitialState['Amplitude'].loc[0])

Como comprobamos la puerta $D_0$ cumple su cometido!

#### 2.2.2 Implementación Circuital $\hat{U}_{|\Psi}\rangle$

Nos queda implementar el Difusor:
$$\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle \langle \Psi|$$

Como ya tenemos implementado

$$\hat{D}_{0} = \hat{I}-2|0\rangle \langle0|$$

y sabemos que:

$$|\Psi\rangle_{n+1}=\hat{R_{n+1}}\hat{P_{n}}|0\rangle_{n+1}$$

La forma rápida de implementar el operador Difusor: $\hat{U}_{|\Psi\rangle }$

$$\hat{U}_{|\Psi\rangle} = \hat{R_{n+1}}\hat{P_{n}}\hat{D}_{0} \hat{P_{n}}^{\dagger} \hat{R_{n+1}}^{\dagger}$$

Este operador se puede interpretar como una reflexión en torno al estado perpendicular a $|\Psi\rangle$


In [ ]:
from QuantumMultiplexors_Module import LoadP_Gate

In [ ]:
P_gate = LoadP_Gate(p_X)

In [ ]:
%qatdisplay P_gate

In [ ]:
from qat.lang.AQASM import Program
qprog = Program()
qbits = qprog.qalloc(n_qbits)
qprog.apply(P_gate, qbits)
qprog.apply(P_gate.dag(), qbits)

In [ ]:
circuit = qprog.to_circ()
%qatdisplay circuit --depth 0

In [ ]:
from qat.lang.AQASM import QRoutine, AbstractGate, RY
from AuxiliarFunctions import TestBins, LeftConditionalProbability
from QuantumMultiplexors_Module import  multiplexor_RY_m
def LoadProbability_Gate(ProbabilityArray):
    """
    Given a discretized probability array the function creates a AbstracGate that allows the load
    of the probability in a Quantum State. The number of qbits of the gate will be log2(len(ProbabilityArray))
    Inputs:
        * ProbabilityArray: np.array. Discretized arrray with the probability to load
    Outuput:
        * P_gate: Abstract Gate. Gate for loading Input probability in a quantum state
    """
    
    #Number of Input qbits for the QWuantum Gate
    #nqbits_ = np.log2(len(ProbabilityArray))
    ##Probability array must have a dimension of 2^n.
    #Condition = (nqbits_%2 ==0) or (nqbits_%2 ==1)
    #assert Condition, 'Length of the ProbabilityArray must be of dimension 2^n with n a int. In this case is: {}.'.format(nqbits_)
    #
    #nqbits = int(nqbits_)
    #nbins = len(ProbabilityArray)
    nqbits = TestBins(ProbabilityArray, text='Function')

    
    P = AbstractGate("P", [int])
    def P_generator(nqbits):
        rout = QRoutine()
        reg = rout.new_wires(nqbits)
        # Now go iteratively trough each qubit computing the probabilities and adding the corresponding multiplexor
        for m in range(nqbits):


            #Calculates Conditional Probability
            ConditionalProbability = LeftConditionalProbability(m, ProbabilityArray)

            
            #Rotation angles: length: 2^(i-1)-1 and i the number of qbits of the step
            thetas = 2.0*(np.arccos(np.sqrt(ConditionalProbability)))
            """
            n_parts = 2**(m+1) #Compute the number of subzones which the current state is codifying
            edges = np.array([a+(b-a)*(i)/n_parts for i in range(n_parts+1)]) #Compute the edges of that subzones
        
            # Compute the probabilities of each subzone by suming the probabilities of the original histogram.
            # There is no need to compute integrals since the limiting accuracy is given by the original discretization.
            # Moreover, this approach allows to handle non analytical probability distributions, measured directly from experiments
            p_zones = np.array([np.sum(ProbabilityArray[np.logical_and(CentersArray>edges[i],CentersArray<edges[i+1])]) for i in range(n_parts)])
            # Compute the probability of standing on the left part of each zone 
            p_left = p_zones[[2*j for j in range(n_parts//2)]]
            # Compute the probability of standing on each zone (left zone + right zone)
            p_tot = p_left + p_zones[[2*j+1 for j in range(n_parts//2)]]
            
            # Compute the rotation angles
            thetas = 2.0*np.arccos(np.sqrt(p_left/p_tot))
            """

            if m == 0:
                # In the first iteration it is only needed a RY gate
                rout.apply(RY(thetas[0]), reg[0])
            else:
                # In the following iterations we have to apply multiplexors controlled by m qubits
                # We call a function to construct the multiplexor, whose action is a block diagonal matrix of Ry gates with angles theta
                multiplexor_RY_m(rout, reg, thetas, m, m)
        return rout
    P.set_circuit_generator(P_generator)
    P_gate = P(nqbits)
    return P_gate

def LoadIntegralFunction_Gate(FunctionArray):
    """
    Load the values of the function f on the states in which the value of the auxiliary qubit is 1 once the probabilities are already loaded.
    The number of the qbits of the gate will be log2(len(FunctionArray)) + 1. This is mandatory. The integral will be loaded in last qbit
    
    Inputs:
        * FunctionArray: np.array. Discretized arrray with the function for integral loading
    Outputs:
        * R_gate (ParamGate) : gate that loads the function into the amplitudes
    """
    assert np.all(FunctionArray<=1.), 'The image of the function must be less than 1. Rescaling is required'
    assert np.all(FunctionArray>=0.), 'The image of the function must be greater than 0. Rescaling is required'
    assert isinstance(FunctionArray, np.ndarray), 'the output of the function p must be a numpy array'

    nqbits = TestBins(FunctionArray, text='Function')
    thetas = 2.0*np.arcsin(np.sqrt(FunctionArray))

    R = AbstractGate("R", [int])# + [float for theta in thetas])
    def R_generator(nqbits):#, *thetas):
        rout = QRoutine()
        reg = rout.new_wires(nqbits+1)
        multiplexor_RY_m(rout, reg, thetas, nqbits, nqbits)
        return rout
    R.set_circuit_generator(R_generator)
    R_gate = R(nqbits)
    return R_gate


In [ ]:
P_g = LoadProbability_Gate(p_X)

In [ ]:
from qat.lang.AQASM import Program
qprog = Program()
qbits = qprog.qalloc(n_qbits)
qprog.apply(P_g, qbits)
qprog.apply(P_g.dag(), qbits)

In [ ]:
circuit = qprog.to_circ()
%qatdisplay circuit --depth 0


In [ ]:
def LoadProbability_Gate(ProbabilityArray):
    """
    Given a discretized probability array the function creates a AbstracGate that allows the load
    of the probability in a Quantum State. The number of qbits of the gate will be log2(len(ProbabilityArray))
    Inputs:
        * ProbabilityArray: np.array. Discretized arrray with the probability to load
    Outuput:
        * P_gate: Abstract Gate. Gate for loading Input probability in a quantum state
    """
    
    #Number of Input qbits for the QWuantum Gate
    #nqbits_ = np.log2(len(ProbabilityArray))
    ##Probability array must have a dimension of 2^n.
    #Condition = (nqbits_%2 ==0) or (nqbits_%2 ==1)
    #assert Condition, 'Length of the ProbabilityArray must be of dimension 2^n with n a int. In this case is: {}.'.format(nqbits_)
    #
    #nqbits = int(nqbits_)
    #nbins = len(ProbabilityArray)
    nqbits = TestBins(ProbabilityArray, text='Function')

    
    P = AbstractGate("P", [int])
    def P_generator(nqbits):
        rout = QRoutine()
        reg = rout.new_wires(nqbits)
        # Now go iteratively trough each qubit computing the probabilities and adding the corresponding multiplexor
        for m in range(nqbits):


            #Calculates Conditional Probability
            ConditionalProbability = LeftConditionalProbability(m, ProbabilityArray)

            
            #Rotation angles: length: 2^(i-1)-1 and i the number of qbits of the step
            thetas = 2.0*(np.arccos(np.sqrt(ConditionalProbability)))
            """
            n_parts = 2**(m+1) #Compute the number of subzones which the current state is codifying
            edges = np.array([a+(b-a)*(i)/n_parts for i in range(n_parts+1)]) #Compute the edges of that subzones
        
            # Compute the probabilities of each subzone by suming the probabilities of the original histogram.
            # There is no need to compute integrals since the limiting accuracy is given by the original discretization.
            # Moreover, this approach allows to handle non analytical probability distributions, measured directly from experiments
            p_zones = np.array([np.sum(ProbabilityArray[np.logical_and(CentersArray>edges[i],CentersArray<edges[i+1])]) for i in range(n_parts)])
            # Compute the probability of standing on the left part of each zone 
            p_left = p_zones[[2*j for j in range(n_parts//2)]]
            # Compute the probability of standing on each zone (left zone + right zone)
            p_tot = p_left + p_zones[[2*j+1 for j in range(n_parts//2)]]
            
            # Compute the rotation angles
            thetas = 2.0*np.arccos(np.sqrt(p_left/p_tot))
            """

            if m == 0:
                # In the first iteration it is only needed a RY gate
                rout.apply(RY(thetas[0]), reg[0])
            else:
                # In the following iterations we have to apply multiplexors controlled by m qubits
                # We call a function to construct the multiplexor, whose action is a block diagonal matrix of Ry gates with angles theta
                multiplexor_RY_m(rout, reg, thetas, m, m)
        return rout
    P.set_circuit_generator(P_generator)
    P_gate = P(nqbits)
    return P_gate

In [ ]:
P_gate = LoadP_Gate({'array':p_X})

In [ ]:
from qat.lang.AQASM import Program
qprog = Program()
qbits = qprog.qalloc(n_qbits)
qprog.apply(P_gate, qbits)
qprog.apply(P_gate.dag(), qbits)

In [ ]:
circuit = qprog.to_circ()
%qatdisplay circuit --depth 0

In [ ]:
LoadP_Gate = AbstractGate("P", [dict])
LoadP_Gate.set_circuit_generator(P_generatorQM)

In [ ]:
from qat.lang.AQASM import QRoutine, AbstractGate, RY
from AuxiliarFunctions import TestBins, LeftConditionalProbability
from QuantumMultiplexors_Module import  multiplexor_RY_m
def P_generatorQM(*ProbabilityArray):
    """
    Function generator for the AbstractGate that allows the loading of a discretized Probability
    in a Quantum State using Quantum Multiplexors
    Inputs:
        * ProbabilityArray: dict. Python dictionary whit a key named "array" whose corresponding item is a numpy array with the discretized
    probability to load. If ProbabilityArray = Dictionary['array']. The number of qbits will be log2(len(ProbabilityArray)). 
    Outuput:
        * qrout: Quantum routine. Routine for loading the discrete probability with Quantum Multiplexors.
    """
    
    
    #ProbabilityArray = Dictionary['array']
    nqbits = TestBins(ProbabilityArray, text='Function')
    
    qrout = QRoutine()
    reg = qrout.new_wires(nqbits)
    # Now go iteratively trough each qubit computing the probabilities and adding the corresponding multiplexor
    for m in range(nqbits):
        #Calculates Conditional Probability
        ConditionalProbability = LeftConditionalProbability(m, ProbabilityArray)        
        #Rotation angles: length: 2^(i-1)-1 and i the number of qbits of the step
        thetas = 2.0*(np.arccos(np.sqrt(ConditionalProbability)))   
        
        if m == 0:
            # In the first iteration it is only needed a RY gate
            qrout.apply(RY(thetas[0]), reg[0])
        else:
            # In the following iterations we have to apply multiplexors controlled by m qubits
            # We call a function to construct the multiplexor, whose action is a block diagonal matrix of Ry gates with angles theta
            multiplexor_RY_m(qrout, reg, thetas, m, m)        
    return qrout  





In [ ]:
LoadP_Gate = AbstractGate("P", [float for i in p_X], circuit_generator=P_generatorQM, arity = int(np.log2(len(p_X))))

In [ ]:
P_gate = LoadP_Gate(*p_X)

In [ ]:
%qatdisplay P_gate

In [ ]:
from QuantumMultiplexors_Module import LoadP_Gate, LoadR_Gate
P_gateD = LoadP_Gate({'array':p_X})
R_gateD = LoadR_Gate({'array':f_X})

In [ ]:
%qatdisplay P_gateD

In [ ]:
from qat.lang.AQASM import Program
qprog = Program()
qbits = qprog.qalloc(n_qbits)
qprog.apply(P_gate, qbits)
qprog.apply(P_gate, qbits)

In [ ]:
circuit = qprog.to_circ()
%qatdisplay circuit --depth 0

In [ ]:
P_gateD.conj()

In [ ]:
"""

LoadP_Gate = AbstractGate(
    "P_Gate",
    [dict],
    circuit_generator = P_generatorQM,
)    

from qat.lang.AQASM import QRoutine, AbstractGate, RY
from QuantumMultiplexors_Module import  multiplexor_RY_m
def R_generatorQM(Dictionary):
    """
    Function generator for creating an AbstractGate that allows the loading of the integral of a given
    discretized function array into a Quantum State using Quantum Multiplexors
    Inputs:
        * Dictionary: dict. Python dictionary with a key named "array" whose corresponding item is a numpy array with the discrietized function. If the discretized function is FunctionArray = Dictionary['array'] the number of qbits will be log2(len(FunctionArray)) + 1 qbits.
    Outuput:
        * qrout: quantum routine. Routine for loading the input function as a integral on the last qbit using Quantum Multiplexors
    """

    FunctionArray = Dictionary['array']

    assert np.all(FunctionArray<=1.), 'The image of the function must be less than 1. Rescaling is required'
    assert np.all(FunctionArray>=0.), 'The image of the function must be greater than 0. Rescaling is required'
    assert isinstance(FunctionArray, np.ndarray), 'the output of the function p must be a numpy array'

    nqbits = TestBins(FunctionArray, text='Function')
    #Calculation of the rotation angles
    thetas = 2.0*np.arcsin(np.sqrt(FunctionArray))


    qrout = QRoutine()
    reg = qrout.new_wires(nqbits+1)
    multiplexor_RY_m(qrout, reg, thetas, nqbits, nqbits)
    return qrout

LoadR_Gate = AbstractGate(
    "R_Gate",
    [dict],
    circuit_generator = R_generatorQM,
    #arity = lambda x:TestBins(x['array'], 'Function')+1
)
"""



In [ ]:
P_gate = LoadP_Gate({'array':p_X})

In [ ]:
from qat.lang.AQASM import Program
qprog = Program()
qbits = qprog.qalloc(n_qbits)
qprog.apply(P_gate, qbits)
#qprog.apply(R_gate.dag(), qbits)
#qprog.apply(R_gate.dag(), qbits)
qprog.apply(P_gate, qbits)

In [ ]:
circuit = qprog.to_circ()
%qatdisplay circuit --depth 0x


In [ ]:
from qat.lang.AQASM import QRoutine, AbstractGate
def U_Phi_generator(nqbits, GateDictionary):
    """
    Circuit generator for the U_Phi_Gate.
    Operation to be implemented: R*P*D_0*P^{+}R^{+}
    Inputs:
        * nqbits: int. Number of Qbits for the circuit
    Outputs:
        * qrout: quantum routine with the circuit implementation
    """
    
    R_gate = GateDictionary['R_gate']
    P_gate = GateDictionary['P_gate']
    
    qrout = QRoutine()
    qbits = qrout.new_wires(nqbits)
    qrout.apply(R_gate.dag(), qbits)
    #qrout.apply(P_gate.dag(), qbits[:-1])
    #D_0 = LoadD0_Gate(len(qbits))
    #qrout.apply(D_0, qbits)
    #qrout.apply(P_gate, qbits[:-1])
    qrout.apply(R_gate, qbits)
    return qrout

LoadUPhi_Gate = AbstractGate(
    "U_Phi", 
    [int, dict],
    circuit_generator = U_Phi_generator,
    arity = lambda x, y: x
)

In [ ]:
qU = LoadingData(p_X, f_X)
P_gate = LoadP_Gate({'array':p_X})
R_gate = LoadR_Gate({'array':f_X})
a = dict({'P_gate': P_gate, 'R_gate': R_gate})


In [ ]:
Op = LoadUPhi_Gate(n_qbits+1, a)

In [ ]:
from expectation_module import load_probabilities, load_function, load_U0, load_U1, load_Q

In [ ]:
nqbits=4
centers, probs, P_gate = load_probabilities(nqbits, p, 0, 1)
R_gate, y = load_function(centers, f, nqbits)

In [ ]:
from qat.lang.AQASM import Program, RY, CNOT, H, AbstractGate, QRoutine, SWAP, PH, I

In [ ]:
qU = LoadingData(p_X, f_X)
P_gate = LoadP_Gate({'array':p_X})
R_gate = LoadR_Gate({'array':f_X})
a = dict({'P_gate': P_gate, 'R_gate': R_gate})

In [ ]:
Phi = LoadUPhi_Gate(a)

In [ ]:
%qatdisplay P_gate

In [ ]:
P_gate.arity

In [ ]:
qU = LoadingData(p_X, f_X)
P_gate = LoadP_Gate({'array':p_X})
R_gate = LoadR_Gate({'array':f_X})
a = dict({'P_gate': P_gate, 'R_gate': R_gate})

In [ ]:
from PhaseAmplification_Module import D_0
from qat.lang.AQASM import QRoutine, AbstractGate
def U_Phi_Gate(nqbits, P_gate, R_gate):
    """
    Create gate U_Phi mandatory for Phase Amplification Algorithm.
    The operator to implement is: I-2|Phi_{n-1}><Phi_{n-1}|. 
    Where the state |Phi_{n-1}> is: |Phi_{n-1}>=R*P*|0_{n+1}>. 
    Where R and P are the gates to load the integral of a function f(x) and 
    the load of a distribution probabilitiy p(x) respectively.
    Inputs:
        * nqbits: int. Number of Qbits of the Gate
        * P_gate: quantum gate for loading probability distribution.
        * R_gate: quantum gate for loading integral of a function f(x)
    Outputs:
        * U_Phi: guantum gate that implements U_Phi gate
    """
    

    def U_Phi_generator(nqbits):
        """
        Circuit generator for the U_Phi_Gate.
        Operation to be implemented: R*P*D_0*P^{+}R^{+}
        Inputs:
            * nqbits: int. Number of Qbits for the circuit
        Outputs:
            * qrout: quantum routine with the circuit implementation
        """
        qrout = QRoutine()
        qbits = qrout.new_wires(nqbits)
        qrout.apply(R_gate.dag(), qbits)
        #qrout.apply(P_gate.dag(), qbits[:-1])
        #qrout.apply(D_0(nqbits), qbits)
        #qrout.apply(P_gate, qbits[:-1])
        #qrout.apply(R_gate, qbits)
        return qrout
    U_Phi = AbstractGate("UPhi", [int])
    U_Phi.set_circuit_generator(U_Phi_generator)
    return U_Phi(nqbits)


In [ ]:
from QuantumMultiplexors_Module import LoadP_Gate, LoadR_Gate

In [ ]:
P_gate = LoadP_Gate({'array':p_X})
R_gate = LoadR_Gate({'array':f_X})

In [ ]:
n_qbits

In [ ]:
from qat.lang.AQASM import Program
qprog = Program()
qbits = qprog.qalloc(n_qbits+1)
#qprog.apply(P_gate, qbits[:-1])
#qprog.apply(R_gate, qbits)
qprog.apply(R_gate.dag(), qbits)
qprog.apply(P_gate.dag(), qbits[:-1])
qprog.apply(D_0(n_qbits+1), qbits)
qprog.apply(P_gate, qbits[:-1])

In [ ]:
circuit = qprog.to_circ()

In [ ]:
%qatdisplay circuit

In [ ]:
qU = LoadingData(p_X, f_X)

In [ ]:
qU.qbit_count

In [ ]:
uphi = U_Phi_Gate(qU.qbit_count, P_gate, R_gate)

In [ ]:
qU.apply(uphi, qU.registers)

In [ ]:
circuitqU = qU.to_circ()

In [ ]:
P_gate

In [ ]:
R_gate

In [ ]:
uphi

In [ ]:
uphi

In [ ]:
%qatdisplay uphi

In [ ]:
U_Phi_Gate(nqbits, P_gate, R_gate)

In [ ]:
from QuantumMultiplexors_Module import LoadP_Gate, LoadR_Gate

In [ ]:
U_Phi_gate = LoadUPhi_Gate(a)

In [ ]:
%qatdisplay  U_Phi_gate --depth 1

In [ ]:
a['P_gate'].arity

In [ ]:
circuitqU = qU.to_circ()

In [ ]:
U_Phi_gate = LoadUPhi_Gate({'P_gate': P_gate, 'R_gate': R_gate})
qU.apply(U_Phi_gate, qU.registers)
circuitqU = qU.to_circ()

In [ ]:
%qatdisplay circuitqU

In [ ]:
from qat.lang.AQASM import AbstractGate, QRoutine
def LoadDifusorGate(nqbits, P_gate, R_gate):

    def U_Phi_generator(nqbits):
        qrout = QRoutine()
        qbits = qrout.new_wires(nqbits)
        qrout.apply(R_gate.dag(), qbits)
        qrout.apply(P_gate.dag(), qbits[:-1])
        qrout.apply(D_0(nqbits), qbits)
        qrout.apply(P_gate, qbits[:-1])
        qrout.apply(R_gate, qbits)
        return qrout
    U_Phi = AbstractGate("UPhi", [int])
    U_Phi.set_circuit_generator(U_Phi_generator)
    return U_Phi(nqbits)

In [ ]:
from QuantumMultiplexors_Module import LoadProbability_Gate, LoadIntegralFunction_Gate

In [ ]:
P_gate = LoadProbability_Gate(p_X)
R_gate = LoadIntegralFunction_Gate(f_X)

In [ ]:
U_gate = LoadDifusorGate(n_qbits+1, P_gate, R_gate)

In [ ]:
qU = LoadingData(p_X, f_X)
qU.apply(U_gate, qU.registers)

In [ ]:
circuitqU = qU.to_circ()
%qatdisplay circuitqU 

In [ ]:
job = circuitqU.to_job()
result = lineal_qpu.submit(job)
qUState = PostProcessResults(result.join())

Cabe destacar que:
$$\hat{U}_{|\Psi\rangle } |\Psi\rangle = (\hat{I} - 2|\Psi\rangle \langle\Psi|)|\Psi\rangle=-|\Psi\rangle$$

Esto lo podemos utilizar para verificar que hemos implementado bien la puerta

In [ ]:
qUState

In [ ]:
InitialState

In [ ]:
#number of Qbits for the circuit
n_qbits = 4
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

In [ ]:
from expectation_module import load_U1

In [ ]:
qU_J = LoadingData(p_X, f_X)
U1_J_gate = load_U1(n_qbits, P_gate, R_gate)
qU_J.apply(U1_J_gate, qU_J.registers)

In [ ]:
circuitqU_J = qU_J.to_circ()
%qatdisplay circuitqU_J 

In [ ]:
job = circuitqU_J.to_job()
result = lineal_qpu.submit(job)
qU_J_State = PostProcessResults(result.join())

In [ ]:
qU_J_State.head()

In [ ]:
InitialState.head()

#### Testeo Implementación Juan

In [ ]:
from qat.lang.AQASM import Program
from expectation_module import load_probabilities, load_function, load_U0, load_U1, load_Q

centers, probs, P_gate = load_probabilities(n_qbits, p, LowerLimit, UpperLimit)
R_gate, y = load_function(centers, f, n_qbits)
Q_gate, U0_gate, U1_gate  = load_Q(n_qbits, P_gate, R_gate)

qU_J = Program()
qbits = qU_J.qalloc(n_qbits+1)
qU_J.apply(P_gate, qbits[:-1])
qU_J.apply(R_gate, qbits)
qU_J.apply(U1_gate, qbits)

In [ ]:
circuitqU_J = qU_J.to_circ()
%qatdisplay circuitqU_J 

In [ ]:
job = circuitqU_J.to_job()
result = lineal_qpu.submit(job)
qU_J_State = PostProcessResults(result.join())

In [ ]:
qU_J_State